In [ ]:
import requests
import pandas as pd
from datetime import datetime
import dapla as dp
import random

In [ ]:
# parametre til KLASS-kall
# 131 kommuner
klasstabell = 131
klassdato = datetime.now().strftime("%Y-%m-%d")

In [ ]:
# KLASS-kall:returnerer gyldige koder for enklassifikasjon på et gitt tidspunkt
response_json = requests.get(f'https://data.ssb.no/api/klass/v1/classifications/{klasstabell}/codesAt.json?date={klassdato}').json()

In [ ]:
# Lager et Pandas-datasett for å se på kodene
pd_response_json = pd.DataFrame(response_json['codes'])
pd_response_json

In [ ]:
# Lager en liste av kodene i klassifikasjonen, til bruk i random genereringen av testdata
kommunekoder_liste = pd_response_json['code'].tolist()
kommunekoder_liste

In [ ]:
random.seed(20)

r = random.sample(range(10000000000), 10)
fnr = [str(x) for x in r]
land = ['000', '346', '106']
# kommnr = ['0301', '1103', '4601', '5401', '5001']
kommnr = kommunekoder_liste
testdata = [
            {"personidentifikator": random.choice(fnr),
             "foedland": random.choice(land),
             "kjoenn": random.randrange(1, 3),
             "kommnr": random.choice(kommnr),
             "mor_personidentifikator": random.choice(fnr)
             }
            for _ in range(10)
            ]
print(testdata)

In [ ]:
random.seed(20)
aar = ['1980', '1960', '2001']
husholdningsnummer = ['12340301', '23451103', '56784601', '56785401', '88885001']
testdata1 = [
            {"personidentifikator": random.choice(range(10000000000,99999999999)),# 11 siffer
             "aargang": random.choice(aar),
             "kjoenn": random.randrange(1, 3),
             "husnr": random.choice(husholdningsnummer),
             #"Yrkesinntekter": (random.sample(range(1000000), 1)),
             "Yrkesinntekter": (random.choice(range(1000000))),
             "kommnr": random.choice(kommnr),
             }
            for _ in range(10)
            ]
print(testdata1)

In [ ]:
df = pd.DataFrame(testdata1)
# pyspark
df_spark = spark.createDataFrame(df)
df_spark.show()

In [ ]:
# Lager et Pandas-datasett for å se på kodene
pd_testdatasett = pd.DataFrame(testdata)
pd_testdatasett

In [ ]:
%document -f pd_testdatasett.json df_spark

In [ ]:
%%output
/felles/metadata_test/testdata/generert_testdata_sample

In [ ]:
df_spark.write\
    .option("valuation", "INTERNAL")\
    .option("state", "INPUT")\
    .path('/felles/metadata_test/testdata/generert_testdata_sample')

In [ ]:
ll =  spark.read.path('/felles/metadata_test/*')
ll.show(100, False)

In [ ]:
help(dp.spark)

In [ ]:
dp.details('/felles/metadata_test/testdata/generert_testdata_sample')